# System utilities

> Utilities to ease the use of operation system commands and files and folders

In [ ]:
#| default_exp ossys

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import re
import unicodedata
import tomllib
from pathlib import Path
from typing import Optional
from fastcore.test import test_eq

## Get project root

In [ ]:
#| export
def get_project_root() -> Optional[Path]:
    """Get the project root directory from either notebook or module context"""
    try:
        try:
            get_ipython()
            current = Path.cwd()
        except NameError:
            current = Path(__file__).resolve().parent

        while current != current.parent: # Stop at root directory
            if (current / 'pyproject.toml').exists():
                return current
            current = current.parent
        raise FileNotFoundError("Could not find pyproject.toml in any parent directory")
    except Exception as e:
        print(f"Error finding project root: {str(e)}")
        return None

#### Example usage

In [ ]:
get_project_root()

Path('/home/jelle/code/hopsa')

## Get project name

In [ ]:
#| export
def get_project_name():
    try:
        toml_file = get_project_root() / "pyproject.toml"
        with open(toml_file, "rb") as f:
            pyproject = tomllib.load(f)
            return pyproject.get("project", {}).get("name", "app")
    except (FileNotFoundError, KeyError):
        # Default to 'app' if file not found or missing project name
        return "app"

#### Example usage

In [ ]:
get_project_name()

'hopsa'

## Sanitize names

Remove special characters from names, convert to lowercase, and remove leading and trailing whitespace. Especially convenient to create filenames and foldernames that are compatible with most operating systems, but can also be used to sanitize names for other purposes.

In [ ]:
#| export
def sanitize_name(name: str) -> str:
    """Remove special characters from names, convert to lowercase, and remove leading and trailing whitespace."""
    nfkd_form = unicodedata.normalize('NFKD', name)
    ascii_name = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])
    return re.sub(r'_+', '_', re.sub(r'[^a-z0-9]', '_', ascii_name.lower()).strip('_'))


In [ ]:
"""Test basic sanitization cases"""
test_eq(sanitize_name("My Test Name"), "my_test_name")
test_eq(sanitize_name("  Trim Me  "), "trim_me")
test_eq(sanitize_name("UPPER CASE"), "upper_case")

"""Test with special characters"""
test_eq(sanitize_name("File@Name#123"), "file_name_123")
test_eq(sanitize_name("user@domain.com"), "user_domain_com")
test_eq(sanitize_name("test$%^&*()name"), "test_name")

"""Test with emoticons and unicode characters"""
test_eq(sanitize_name("Happy 😊 Face"), "happy_face")
test_eq(sanitize_name("Thumbs 👍 Up"), "thumbs_up")
test_eq(sanitize_name("Café "), "cafe")
test_eq(sanitize_name("Mötörhead"), "motorhead")

"""Test with multiple spaces and various whitespace characters"""
test_eq(sanitize_name("Too    Many   Spaces"), "too_many_spaces")
test_eq(sanitize_name("New\nLine"), "new_line")
test_eq(sanitize_name("Tab\tSeparated"), "tab_separated")

"""Test edge cases and empty inputs"""
test_eq(sanitize_name(""), "")
test_eq(sanitize_name(" "), "")
test_eq(sanitize_name("!@#$%^"), "")
test_eq(sanitize_name("123"), "123")
test_eq(sanitize_name("a"), "a")


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()